In [1]:
import pandas as pd
x_train = pd.read_csv(r'C:\Users\brian\OneDrive\Desktop\JUNSOO\bigdata\data\titanic_x_train.csv' , encoding='cp949')
y_train = pd.read_csv(r'C:\Users\brian\OneDrive\Desktop\JUNSOO\bigdata\data\titanic_y_train.csv' , encoding='cp949')
x_test = pd.read_csv(r'C:\Users\brian\OneDrive\Desktop\JUNSOO\bigdata\data\titanic_x_test.csv' , encoding='cp949')

In [2]:
#x_train.info()
#print(x_train.describe().T)
print(y_train.info())

# 각 series 의 unique 함수를 통해 
# 성별, 티켓등급 ==> 범주형
# 승객이름, 티켓번호, 객실번호 ==> string data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   癤풮assengerId  891 non-null    int64
 1   Survived      891 non-null    int64
dtypes: int64(2)
memory usage: 14.1 KB
None


In [3]:
## 독립변수와 종속변수 관계 확인하기

# data프레임 만들기 axis = 1 이므로 세로방향 통합
data = pd.concat([x_train , y_train] , axis = 1)        #axis = 1 세로방향 통합

# 성별 칼럼에 따른 생존여부의 평균을 계산
print(data.groupby(['성별'])['Survived'].mean())        #groupby 문법

# 티켓 등급에 따른 생존여부의 평균을 계산
print(data.groupby(['티켓등급'])['Survived'].mean())

# 선착장에 따른 생존여부 평균을 계산
print(data.groupby(['선착장'])['Survived'].mean())

성별
female    0.742038
male      0.188908
Name: Survived, dtype: float64
티켓등급
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64
선착장
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64


In [4]:
x_train.info()
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   티켓등급         891 non-null    int64  
 2   승객이름         891 non-null    object 
 3   성별           891 non-null    object 
 4   나이           714 non-null    float64
 5   형제자매배우자수     891 non-null    int64  
 6   부모자식수        891 non-null    int64  
 7   티켓번호         891 non-null    object 
 8   운임요금         891 non-null    float64
 9   객실번호         204 non-null    object 
 10  선착장          889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   癤풮assengerId  891 non-null    int64
 1   Survived      891 non-null    int64
dtypes: int64(2)
m

In [5]:
## 데이터 전처리

# 예측에 불필요한 데이터 // 최종 제출을 위해 저장
x_test_passenger_id = x_test['PassengerId']
x_train.drop(columns = 'PassengerId' , inplace = True)
y_train.drop(columns = '癤풮assengerId' , inplace = True)
x_test.drop(columns = 'PassengerId' , inplace = True)



In [6]:
# 독립변수에서 티켓번호가 중복된 값이 50%를 넘어감 ==> 삭제
x_test.drop(columns = '티켓번호' , inplace = True)
x_train.drop(columns = '티켓번호' , inplace = True)

#승객이름도 삭제
x_test.drop(columns = '승객이름' , inplace = True)
x_train.drop(columns = '승객이름' , inplace = True)

In [7]:
## 결측치 처리하기
x_train.isnull().sum()

# 나이 177 / 객실번호 687 / 선착장 2

# 나이와 생존칼럼간의 상관관계파악 concat 한 data 셋에서 확인
print(data[['나이' , 'Survived']].corr())       #상관관계까 -0.077~ 이므로 나이칼럼도 삭제

x_train.drop(columns = '나이' , inplace = True)
x_test.drop(columns = '나이' , inplace = True)

                나이  Survived
나이        1.000000 -0.077221
Survived -0.077221  1.000000


In [8]:
# 객실번호 역시 데이터간 값이 상이하고 결측치 처리가 어려움 ==> 삭제
x_train.drop(columns = '객실번호' , inplace = True)
x_test.drop(columns = '객실번호' , inplace = True)

In [9]:
# 선착장 칼럼별로 갯수세기
print(x_train.groupby(['선착장'])['선착장'].count())

# 결측치를 S로 대치하기
x_train['선착장'].fillna('S' , inplace = True)




선착장
C    168
Q     77
S    644
Name: 선착장, dtype: int64


C:\Users\brian\AppData\Local\Temp\ipykernel_6580\2041752345.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x_train['선착장'].fillna('S' , inplace = True)


In [10]:
## 범주형 변수를 인코딩하기 ==>> 이부분 개념 다시 잡기 *****

# 독립변수 데이터셋의 범주형 변수인 '성별'를 연속형 변수로 변환하기
# male = 0 , female = 1
x_train['성별'].replace('male', 0 , inplace = True)
x_train['성별'].replace('female' , 1 , inplace = True)

x_test['성별'] = x_test['성별'].replace('male' , 0)
x_test['성별'] = x_test['성별'].replace('female' , 1)


C:\Users\brian\AppData\Local\Temp\ipykernel_6580\2436389143.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x_train['성별'].replace('male', 0 , inplace = True)
C:\Users\brian\AppData\Local\Temp\ipykernel_6580\2436389143.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [11]:
# x_train 선착장 칼럼에대해 원핫 인코딩 수행한후 선착장_dummy에 저장하기

선착장_dummy = (pd
             .get_dummies(x_train['선착장'] , drop_first = True)
             .rename(columns ={'Q':'선착장Q', 'S':'선착장S'})
             )
x_train = pd.concat([x_train , 선착장_dummy] , axis = 1)


# '선착장' 칼럼 삭제하기
x_train.drop(columns = '선착장' , inplace = True)

In [12]:
print(x_train.head())


   티켓등급  성별  형제자매배우자수  부모자식수     운임요금   선착장Q   선착장S
0     3   0         1      0   7.2500  False   True
1     1   1         1      0  71.2833  False  False
2     3   1         0      0   7.9250  False   True
3     1   1         1      0  53.1000  False   True
4     3   0         0      0   8.0500  False   True


In [13]:
# x_test 선착장 칼럼에대해 원핫 인코딩 수행한후 선착장_dummy에 저장하기

선착장_dummy = (pd
             .get_dummies(x_test['선착장'] , drop_first = True)     # get_dummies 함수 문법 및 parameter ***
             .rename(columns = {'Q':'선착장Q' , 'S':'선착장S'})     # rename 함수 문법 및 parameter ***
             )
x_test = pd.concat([x_test , 선착장_dummy] , axis = 1)

#선착장 칼럼 삭제하기
x_test.drop(columns = '선착장' , inplace = True)


In [14]:
print(x_test.head())

   티켓등급  성별  형제자매배우자수  부모자식수     운임요금   선착장Q   선착장S
0     3   0         0      0   7.8292   True  False
1     3   1         1      0   7.0000  False   True
2     2   0         0      0   9.6875   True  False
3     3   0         0      0   8.6625  False   True
4     3   1         1      1  12.2875  False   True


In [15]:
## 파생변수 만들기
# 형제자매배우자수 와 부모자식수 칼럼이 가족이라는 유사한 의미를 지니므로 파생변수 생성
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']

# 유사한 의미를 가졌던 칼럼 삭제
x_train.drop(columns = ['형제자매배우자수' , '부모자식수'] , inplace = True)

In [16]:
# test 데이터에 대해서도 똑같이 수행
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']

# 유사한 의미를 가지는 칼럼삭제
x_test.drop(columns = ['형제자매배우자수' , '부모자식수'] , inplace = True)

# note) train 데이터와 test 데이터를 같은 수행을 해야지 학습이가능하다

In [17]:
### 학습하고 평가하기 pg 441 read ***

# 학습데이터와 테스트 데이터에서
# 학습데이터 (x_train , y_train) 을 분리해서 
# 학습용과 검증용 데이터로 평가를 위한 X_TRAIN,Y_TRAIN AND X_TEST, Y_TEST 데이터를 만듦
# 즉 모델을 학습시키고(X_TRAIN, Y_TRAIN) -> 평가지표 계산하고(X_TEST , Y_TEST) -> x_test로 y_test 구하기! (예측값,확률) 

In [18]:
## 데이터 분리하기 
# (학습데이터에서 평가용 데이터를 분리해내기 x_train => X_TRAIN, X_TEST
#                                         y_train => Y_TRAIN, Y_TEST)


# 데이터 분석을 위한 train_test_split 함수 가져오기

from sklearn.model_selection import train_test_split

# 학습용과 검증용을 8:2로 분류하여 각 4개의 변수에 저장하기

X_TRAIN, X_TEST , Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size = 0.2 , random_state = 10)     #traints 문법..?

print(X_TRAIN.shape)
print(X_TEST.shape)
print(Y_TRAIN.shape)
print(Y_TEST.shape)

(712, 6)
(179, 6)
(712, 1)
(179, 1)


In [19]:
## 데이터 학습 및 하이퍼 파라미터 튜닝하기
# 종속변수 값을 분류하는 모델
from xgboost import XGBClassifier

model = XGBClassifier(eval_metric = 'error', random_state = 10)     #eval_metrice 파라미터..
model.fit(X_TRAIN, Y_TRAIN)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='error', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=10, ...)

In [20]:
## 결과 예측하기
# 종속변수인 Survived 값을 예측하기 위해 모델에 .predict() 함수를 사용하여 값을 예측
# 즉 문제에서 주어진 테스트 데이터(x_test)로 predict를 수행하여 y_test를 예측 하는 것
# predict로 추출된 결과는 데이터프레임 형태가 아니므로 pd.DataFrame() 이용!

y_test_predicted = (pd.DataFrame(model.predict(x_test))
                                      .rename(columns = {0:'Survived'})
                                      )
print(pd.DataFrame(y_test_predicted).head(10))

# 승객이 사망 혹은 생존할 확률 ==> proba
print(pd.DataFrame(model.predict_proba(x_test))[0])     #사망확률
print(pd.DataFrame(model.predict_proba(x_test))[1])     #생존확률

# 학습이 완료된 모델을 통해 Y_TEST 값을 예측하기 : 평가지표 계산용
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST))

   Survived
0         0
1         0
2         0
3         0
4         1
5         0
6         0
7         1
8         1
9         0
0      0.924291
1      0.555195
2      0.875672
3      0.938795
4      0.093296
         ...   
413    0.881565
414    0.004147
415    0.979141
416    0.881565
417    0.402825
Name: 0, Length: 418, dtype: float32
0      0.075709
1      0.444805
2      0.124328
3      0.061205
4      0.906704
         ...   
413    0.118435
414    0.995853
415    0.020859
416    0.118435
417    0.597175
Name: 1, Length: 418, dtype: float32


In [21]:
## 모델 평가하기
# 학습이 완료된 이후에는 채점기준인 roc_auc_score 함수를 이용하여 모델 평가를 수행한다.

# sklearn 패키지의 metrics 모듈에 있는 roc_auc_score 함수를 가져온다.
from sklearn.metrics import roc_auc_score

# 1차 학습 모델의 ROC 평가지표 값을 확인하기
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))      #기본값 모델링 결과

# 2차 학습 모델의 ROC 평가지표 값을 확인하기
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))      #하이퍼 파라미터 적용 결과..?

0.7769506479183899
0.7769506479183899


In [22]:
## 결과 제출하기
# x_test_passenger_id 변수와 y_test_predicted 변수를 세로방향으로 붙히기
print(pd.concat([x_test_passenger_id , y_test_predicted] , axis = 1))

# to_csv 함수를 사용하여 저장
final = pd.concat([x_test_passenger_id , y_test_predicted] , axis = 1)
final.to_csv(r'C:\Users\brian\OneDrive\Desktop\JUNSOO\bigdata\data\1234m.csv' , index = False)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]
